Load HCP data and connectivity analysis.

# Tutorial PCA

# Functions

In [ ]:
def get_sample_cov_matrix(X):
  """
  Returns the sample covariance matrix of data X

  Args:
    X (numpy array of floats) : Data matrix each column corresponds to a
                                different random variable

  Returns:
    (numpy array of floats)   : Covariance matrix
  """

  #################################################
  ## TODO for students: calculate the covariance matrix
  # Fill out function and remove
  #raise NotImplementedError("Student exercise: calculate the covariance matrix!")
  #################################################

  # Subtract the mean of X
  X = X - np.mean(X, 0)

  # Calculate the covariance matrix (hint: use np.matmul)
  cov_matrix = 1 / X.shape[0] * np.matmul(X.T, X)

  return cov_matrix

In [ ]:
def sort_evals_descending(evals, evectors):
  """
  Sorts eigenvalues and eigenvectors in decreasing order. Also aligns first two
  eigenvectors to be in first two quadrants (if 2D).

  Args:
    evals (numpy array of floats)    : Vector of eigenvalues
    evectors (numpy array of floats) : Corresponding matrix of eigenvectors
                                        each column corresponds to a different
                                        eigenvalue

  Returns:
    (numpy array of floats)          : Vector of eigenvalues after sorting
    (numpy array of floats)          : Matrix of eigenvectors after sorting
  """

  index = np.flip(np.argsort(evals))
  evals = evals[index]
  evectors = evectors[:, index]
  if evals.shape[0] == 2:
    if np.arccos(np.matmul(evectors[:, 0],
                           1 / np.sqrt(2) * np.array([1, 1]))) > np.pi / 2:
      evectors[:, 0] = -evectors[:, 0]
    if np.arccos(np.matmul(evectors[:, 1],
                           1 / np.sqrt(2) * np.array([-1, 1]))) > np.pi / 2:
      evectors[:, 1] = -evectors[:, 1]
  return evals, evectors

In [ ]:
def change_of_basis(X, W):
  """
  Projects data onto new basis W.

  Args:
    X (numpy array of floats) : Data matrix each column corresponding to a
                                different random variable
    W (numpy array of floats) : new orthonormal basis columns correspond to
                                basis vectors

  Returns:
    (numpy array of floats)    : Data matrix expressed in new basis
  """

  #################################################
  ## TODO for students: project the data onto a new basis W
  # Fill out function and remove
  #raise NotImplementedError("Student exercise: implement change of basis")
  #################################################

  # Project data onto new basis described by W
  Y = X @ W

  return Y

# Plotting functions

In [ ]:
def plot_eigenvalues(evals):
  """
  Plots eigenvalues.

  Args:
      (numpy array of floats) : Vector of eigenvalues

  Returns:
    Nothing.

  """
  plt.figure(figsize=(4, 4))
  plt.plot(np.arange(1, len(evals) + 1), evals, 'o-k')
  plt.xlabel('Component')
  plt.ylabel('Eigenvalue')
  plt.title('Scree plot')
  plt.xticks(np.arange(1, len(evals) + 1))
  plt.ylim([0, 2.5])
  plt.show()


def plot_data(X):
  """
  Plots bivariate data. Includes a plot of each random variable, and a scatter
  scatter plot of their joint activity. The title indicates the sample
  correlation calculated from the data.

  Args:
    X (numpy array of floats) : Data matrix each column corresponds to a
                                different random variable

  Returns:
    Nothing.
  """

  fig = plt.figure(figsize=[8, 4])
  gs = fig.add_gridspec(2, 2)
  ax1 = fig.add_subplot(gs[0, 0])
  ax1.plot(X[:, 0], color='k')
  plt.ylabel('Neuron 1')
  ax2 = fig.add_subplot(gs[1, 0])
  ax2.plot(X[:, 1], color='k')
  plt.xlabel('Sample Number (sorted)')
  plt.ylabel('Neuron 2')
  ax3 = fig.add_subplot(gs[:, 1])
  ax3.plot(X[:, 0], X[:, 1], '.', markerfacecolor=[.5, .5, .5],
           markeredgewidth=0)
  ax3.axis('equal')
  plt.xlabel('Neuron 1 activity')
  plt.ylabel('Neuron 2 activity')
  plt.title('Sample corr: {:.1f}'.format(np.corrcoef(X[:, 0], X[:, 1])[0, 1]))
  plt.show()


def plot_data_new_basis(Y):
  """
  Plots bivariate data after transformation to new bases. Similar to plot_data
  but with colors corresponding to projections onto basis 1 (red) and
  basis 2 (blue).
  The title indicates the sample correlation calculated from the data.

  Note that samples are re-sorted in ascending order for the first random
  variable.

  Args:
    Y (numpy array of floats) : Data matrix in new basis each column
                                corresponds to a different random variable

  Returns:
    Nothing.
  """

  fig = plt.figure(figsize=[8, 4])
  gs = fig.add_gridspec(2, 2)
  ax1 = fig.add_subplot(gs[0, 0])
  ax1.plot(Y[:, 0], 'r')
  plt.ylabel('Projection \n basis vector 1')
  ax2 = fig.add_subplot(gs[1, 0])
  ax2.plot(Y[:, 1], 'b')
  plt.xlabel('Sample number')
  plt.ylabel('Projection \n basis vector 2')
  ax3 = fig.add_subplot(gs[:, 1])
  ax3.plot(Y[:, 0], Y[:, 1], '.', color=[.5, .5, .5])
  ax3.axis('equal')
  plt.xlabel('Projection basis vector 1')
  plt.ylabel('Projection basis vector 2')
  plt.title('Sample corr: {:.1f}'.format(np.corrcoef(Y[:, 0], Y[:, 1])[0, 1]))
  plt.show()


def plot_basis_vectors(X, W):
  """
  Plots bivariate data as well as new basis vectors.

  Args:
    X (numpy array of floats) : Data matrix each column corresponds to a
                                different random variable
    W (numpy array of floats) : Square matrix representing new orthonormal
                                basis each column represents a basis vector

  Returns:
    Nothing.
  """

  plt.figure(figsize=[4, 4])
  plt.plot(X[:, 0], X[:, 1], '.', color=[.5, .5, .5], label='Data')
  plt.axis('equal')
  plt.xlabel('Neuron 1 activity')
  plt.ylabel('Neuron 2 activity')
  plt.plot([0, W[0, 0]], [0, W[1, 0]], color='r', linewidth=3,
           label='Basis vector 1')
  plt.plot([0, W[0, 1]], [0, W[1, 1]], color='b', linewidth=3,
           label='Basis vector 2')
  plt.legend()
  plt.show()

# PCA

In [ ]:
def pca(X):
  """
  Sorts eigenvalues and eigenvectors in decreasing order.

  Args:
    X (numpy array of floats): Data matrix each column corresponds to a
                               different random variable

  Returns:
    (numpy array of floats)  : Data projected onto the new basis
    (numpy array of floats)  : Vector of eigenvalues
    (numpy array of floats)  : Corresponding matrix of eigenvectors

  """

  #################################################
  ## TODO for students: calculate the covariance matrix
  # Fill out function and remove
  #raise NotImplementedError("Student exercise: sort eigenvalues/eigenvectors!")
  #################################################

  # Calculate the sample covariance matrix
  cov_matrix = get_sample_cov_matrix(X)

  # Calculate the eigenvalues and eigenvectors
  evals, evectors = np.linalg.eigh(cov_matrix)

  # Sort the eigenvalues in descending order
  evals, evectors = sort_evals_descending(evals, evectors)


  # Project the data onto the new eigenvector basis
  score = change_of_basis(X, evectors)

  return score, evectors, evals


# Perform PCA on the data matrix X
#score, evectors, evals = pca(reshaped_matrix)

# Plot the data projected into the new basis
#plot_data_new_basis(score)

# Data load

In [ ]:
from google.colab import drive
import os
import requests
import tarfile

# Montar Google Drive
drive.mount('/content/drive')

# Definir directorios y archivos
drive_dir = "/content/drive/My Drive/HCP_data"
os.makedirs(drive_dir, exist_ok=True)
fnames = ["hcp_rest.tgz", "hcp_covariates.tgz", "atlas.npz"]
urls = ["https://osf.io/bqp7m/download",
        "https://osf.io/x5p4g/download",
        "https://osf.io/j5kuc/download"]

# Función para descargar archivos
def download_file(url, fname):
    try:
        r = requests.get(url)
        r.raise_for_status()  # Check for HTTP errors
    except requests.ConnectionError:
        print("!!! Failed to download data: Connection Error !!!")
    except requests.HTTPError as err:
        print(f"!!! Failed to download data: HTTP Error {err} !!!")
    else:
        print(f"Downloading {fname}...")
        with open(fname, "wb") as fid:
            fid.write(r.content)
        print(f"Download {fname} completed!")

# Descargar los archivos si no existen en Google Drive
for fname, url in zip(fnames, urls):
    drive_path = os.path.join(drive_dir, fname)
    if not os.path.isfile(drive_path):
        download_file(url, drive_path)
    else:
        print(f"{fname} already exists in Google Drive.")

# Si necesitas extraer los archivos
def extract_file(filepath, extract_to):
    if tarfile.is_tarfile(filepath):
        with tarfile.open(filepath) as tar:
            tar.extractall(path=extract_to)
        print(f"Extracted {os.path.basename(filepath)} to {extract_to}")
    else:
        print(f"{os.path.basename(filepath)} is not a valid tar file.")

for fname in fnames:
    drive_path = os.path.join(drive_dir, fname)
    extract_file(drive_path, drive_dir)


Mounted at /content/drive
hcp_rest.tgz already exists in Google Drive.
hcp_covariates.tgz already exists in Google Drive.
atlas.npz already exists in Google Drive.
Extracted hcp_rest.tgz to /content/drive/My Drive/HCP_data


In [ ]:
!pip install nilearn --quiet
import os
import numpy as np
import matplotlib.pyplot as plt

# Necessary for visualization
from nilearn import plotting, datasets

In [ ]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "/content/drive/My Drive/HCP_data"
if not os.path.isdir(HCP_DIR):
  os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasesr parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in sec

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated multiple times in each subject
N_RUNS_REST = 4
N_RUNS_TASK = 2

# Time series data are organized by experiment, with each experiment
# having an LR and RL (phase-encode direction) acquistion
BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL",
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR",
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]

# You may want to limit the subjects used during code development.
# This will use all subjects:
subjects = range(N_SUBJECTS)

In [ ]:
import pandas as pd
language_csv = '/content/drive/My Drive/HCP_data/hcp/behavior/language.csv'
language_df = pd.read_csv(language_csv)
filtered_df = language_df[language_df['ConditionName'] == 'STORY']
filtered_df
run_mean = filtered_df.groupby('Subject')[['ACC', 'AVG_DIFFICULTY_LEVEL', 'MEDIAN_RT']].mean().reset_index()

avg_dif_df = run_mean[['Subject', 'AVG_DIFFICULTY_LEVEL']]

avg_dif_df.head()

#plt.hist(run_mean['ACC'])
plt.hist(run_mean['AVG_DIFFICULTY_LEVEL'])




In [ ]:
# Definir el umbral
threshold = 10.5

# Filtrar los datos por debajo del umbral
below_threshold_df = run_mean[run_mean['AVG_DIFFICULTY_LEVEL'] < threshold]

# Filtrar los datos por encima o iguales al umbral
above_threshold_df = run_mean[run_mean['AVG_DIFFICULTY_LEVEL'] >= threshold]

# Mostrar los resultados
print("Below threshold:")
print(below_threshold_df)

print("\n above the threshold:")
print(above_threshold_df)


In [ ]:
# Crear una nueva columna 'LABEL' basada en la condición del umbral
run_mean['LABEL'] = (run_mean['AVG_DIFFICULTY_LEVEL'] >= threshold).astype(int)

# Filtrar los datos por debajo del umbral
below_threshold_df = run_mean[run_mean['LABEL'] == 0]

# Filtrar los datos por encima o iguales al umbral
above_threshold_df = run_mean[run_mean['LABEL'] == 1]

# Mostrar los resultados
print("Below threshold:")
print(below_threshold_df)

print("\nAbove the threshold:")
print(above_threshold_df)

# MATRIX LOADING

In [ ]:
def get_image_ids(name):
  """Get the 1-based image indices for runs in a given experiment.

    Args:
      name (str) : Name of experiment ("rest" or name of task) to load
    Returns:
      run_ids (list of int) : Numeric ID for experiment image files

  """
  run_ids = [
             i for i, code in enumerate(BOLD_NAMES, 1) if name.upper() in code
             ]
  if not run_ids:
    raise ValueError(f"Found no data for '{name}'")
  return run_ids


def load_timeseries(subject, name, dir,
                    runs=None, concat=True, remove_mean=True):
  """Load timeseries data for a single subject.

  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of experiment ("rest" or name of task) to load
    dir (str) : data directory
    run (None or int or list of ints): 0-based run(s) of the task to load,
      or None to load all runs.
    concat (bool) : If True, concatenate multiple runs in time
    remove_mean (bool) : If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_tp array): Array of BOLD data values

  """
  # Get the list relative 0-based index of runs to use
  if runs is None:
    runs = range(N_RUNS_REST) if name == "rest" else range(N_RUNS_TASK)
  elif isinstance(runs, int):
    runs = [runs]

  # Get the first (1-based) run id for this experiment
  offset = get_image_ids(name)[0]

  # Load each run's data
  bold_data = [
               load_single_timeseries(subject,
                                      offset + run,
                                      dir,
                                      remove_mean) for run in runs
               ]

  # Optionally concatenate in time
  if concat:
    bold_data = np.concatenate(bold_data, axis=-1)

  return bold_data


def load_single_timeseries(subject, bold_run, dir, remove_mean=True):
  """Load timeseries data for a single subject and single run.

  Args:
    subject (int): 0-based subject ID to load
    bold_run (int): 1-based run index, across all tasks
    dir (str) : data directory
    remove_mean (bool): If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_path = os.path.join(dir, "subjects", str(subject), "timeseries")
  bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
  ts = np.load(os.path.join(bold_path, bold_file))
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts


def load_evs(subject, name, condition, dir):
  """Load EV (explanatory variable) data for one task condition.

  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of task
    condition (str) : Name of condition
    dir (str) : data directory

  Returns
    evs (list of dicts): A dictionary with the onset, duration, and amplitude
      of the condition for each run.

  """
  evs = []
  for id in get_image_ids(name):
    task_key = BOLD_NAMES[id - 1]
    ev_file = os.path.join(dir, "subjects", str(subject), "EVs",
                           task_key, f"{condition}.txt")
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev = dict(zip(["onset", "duration", "amplitude"], ev_array))
    evs.append(ev)
  return evs

Extract and concatenate timeseries

In [ ]:
timeseries = load_timeseries(subject=0,
                             name="rest",
                             dir=os.path.join(HCP_DIR, "hcp_rest"),
                             runs=1)
print(timeseries.shape)  # n_parcel x n_timepoint

In [ ]:
timeseries_rest = []
for subject in subjects:
  ts_concat = load_timeseries(subject, name="rest",
                              dir=os.path.join(HCP_DIR, "hcp_rest"))
  timeseries_rest.append(ts_concat)

Construct functional connectivity matrix (FC)

In [ ]:
f_connectivity_matrix = np.zeros((N_SUBJECTS, N_PARCELS, N_PARCELS))
for sub, ts in enumerate(timeseries_rest):
  f_connectivity_matrix[sub] = np.corrcoef(ts)

group_fc = f_connectivity_matrix.mean(axis=0)

plt.figure()
plt.imshow(group_fc, interpolation="none", cmap="bwr", vmin=-1, vmax=1)
plt.colorbar()
plt.show()

In [ ]:
lower_triangle = np.tril(f_connectivity_matrix, k=-1)

In [ ]:
group_fc_lower = lower_triangle.mean(axis=0)
plt.figure()
plt.imshow(lower_triangle[1,:,:], interpolation="none", cmap="bwr", vmin=-1, vmax=1)
plt.colorbar()
plt.show()

# PCA and tSNE



# Screeplot

# TA advise: define a function that thresholds the accuracy. make a vector of n_subjects and assing the label 0 or 1 based on a threshold, (we can have more groups but less group the model will perform better)

In [ ]:
# Función para asignar etiquetas basadas en un umbral
def assign_labels(dataframe, column, threshold):
    labels = (dataframe[column] > threshold).astype(int)
    return labels

# Asignar etiquetas
avg_dif_df['Label'] = assign_labels(avg_dif_df, 'AVG_DIFFICULTY_LEVEL', threshold)

# Mostrar el DataFrame con las etiquetas
print(avg_dif_df)

# Crear un histograma de 'AVG_DIFFICULTY_LEVEL' con etiquetas
plt.hist(avg_dif_df['AVG_DIFFICULTY_LEVEL'], bins=30, alpha=0.5, label='AVG_DIFFICULTY_LEVEL')
plt.axvline(threshold, color='r', linestyle='dashed', linewidth=1, label=f'Threshold = {threshold:.2f}')
plt.xlabel('AVG_DIFFICULTY_LEVEL')
plt.ylabel('Frequency')
plt.title('Histogram of AVG_DIFFICULTY_LEVEL with Threshold')
plt.legend()
plt.show()

# Vector de etiquetas
labels_vector = avg_dif_df['Label'].values
print(labels_vector)

In [ ]:
# Suponiendo que ya has calculado la matriz de conectividad grupal
# group_fc tiene dimensiones (N_PARCELS, N_PARCELS)

# Aplanar la matriz de conectividad funcional para cada sujeto
lower_triangle_indices = np.tril_indices(N_PARCELS, k=-1)
flat_fc = [fc[lower_triangle_indices] for fc in f_connectivity_matrix]

# Convertir a un array de numpy
flat_fc = np.array(flat_fc)

print(flat_fc.shape)  # Debería ser (N_SUBJECTS, N_PARCELS*(N_PARCELS-1)/2)


In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Step 2: Standardize the data
scaler = StandardScaler()
standardized_data = scaler.fit_transform(flat_fc)

# Step 3: Perform PCA
pca = PCA(n_components=50)  # You can adjust the number of components
principal_components = pca.fit_transform(standardized_data)

# Step 4: Visualize the results
plt.figure(figsize=(8, 6))
plt.scatter(principal_components[:, 0], principal_components[:, 1], c='blue')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Subjects using Lower Triangle')
plt.show()

# Explained variance
explained_variance = pca.explained_variance_ratio_


In [ ]:
# # # Explained variance
explained_variance = pca.explained_variance_
explained_variance_ratio = pca.explained_variance_ratio_

# # # Plot eigenvalues (explained variance)
plt.figure(figsize=(10, 6))
plt.plot(explained_variance, marker='o', linestyle='--')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue (Explained Variance)')
plt.title('Eigenvalues of Principal Components')
plt.grid(True)
plt.show()

# # # Plot cumulative explained variance
plt.figure(figsize=(10, 6))
plt.plot(np.cumsum(explained_variance_ratio), marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance by PCA Components')
plt.grid(True)
plt.show()

In [ ]:
# Find the number of components needed to explain 90% of the variance
explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
n_components_90 = np.argmax(explained_variance_ratio >= 0.90) + 1
n_components_75 = np.argmax(explained_variance_ratio >= 0.75) + 1
n_components_50 = np.argmax(explained_variance_ratio >= 0.50) + 1



print(f"Number of components to retain 90% variance: {n_components_90}")
print(f"Number of components to retain 75% variance: {n_components_75}")
print(f"Number of components to retain 50% variance: {n_components_50}")

we can try to do a linear regression to predict the avg_difficult instead of a clasiffier and we won't need the labels

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Supongamos que tienes datos escalados X_scaled

# Aplicar PCA sin especificar el número de componentes
pca = PCA()
pca.fit(standardized_data)

# Obtener la varianza explicada acumulada
explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Graficar la varianza explicada acumulada
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o')
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza Explicada Acumulada')
plt.title('Varianza Explicada Acumulada por Componentes Principales')
plt.grid(True)
plt.axhline(y=0.90, color='r', linestyle='--')  # Línea para 90% de varianza explicada
plt.show()


# The model starts here

In [ ]:
avg_dif_df

In [ ]:
labels_vector = run_mean['LABEL'].values
print(labels_vector)
print(labels_vector.shape)



In [ ]:
principal_components

# try other models
random forest, decision making trees, svm,
feature selection (other than pca)
knearest neighbour


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming principal_components is your feature data and y is your target variable
X = principal_components
y = labels_vector  # Replace with your actual target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42)

# Initialize the classifier
classifier = LogisticRegression(random_state=42)

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

report = classification_report(y_test, y_pred)
print('Classification Report:')
print(report)

matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(matrix)


In [ ]:
print(principal_components.shape)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix
matrix = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classifier.classes_, yticklabels=classifier.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


# Model Validation

k fold validation,

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(classifier, X, y, cv=5)

print(f'Cross-validation scores: {cv_scores}')
print(f'Mean CV score: {cv_scores.mean()}')


# Feature Importance

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Feature importance for logistic regression
if hasattr(classifier, 'coef_'):
    importance = np.abs(classifier.coef_[0])
    feature_names = [f'Feature {i}' for i in range(X.shape[1])]
    feature_importance = sorted(zip(feature_names, importance), key=lambda x: x[1], reverse=True)

    print('Feature Importance:')
    for feature, importance in feature_importance:
        print(f'{feature}: {importance}')

    # Plot feature importance
    feature_names, importances = zip(*feature_importance)
    plt.figure(figsize=(10, 8))
    plt.barh(feature_names, importances, align='center')
    plt.xlabel('Importance')
    plt.title('Feature Importance')
    plt.gca().invert_yaxis()  # Invert y axis to have the most important feature at the top
    plt.show()
else:
    print('The classifier does not have a `coef_` attribute.')

# XGboost


In [ ]:
from xgboost import XGBClassifier

# Initialize the classifier
classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Compute the confusion matrix
matrix = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classifier.classes_, yticklabels=classifier.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


# Grid Search XGboost

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}

# Initialize the classifier
classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters and score
print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best Score: {grid_search.best_score_}')

# Use the best estimator to make predictions
best_classifier = grid_search.best_estimator_
y_pred = best_classifier.predict(X_test)

# Evaluate the classifiergit
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


# elbow kmeans

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Using t-SNE results with perplexity=30 for clustering
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(features)

# Elbow Method
inertia = []
k_values = range(1, 11)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(tsne_result)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(k_values, inertia, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()


# Random forest

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you have X_train, y_train, X_test, and y_test defined

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Silhouette Score

In [ ]:
from sklearn.metrics import silhouette_score

# Silhouette Score Method
silhouette_scores = []

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(tsne_result)
    silhouette_scores.append(silhouette_score(tsne_result, labels))

plt.figure(figsize=(8, 6))
plt.plot(range(2, 11), silhouette_scores, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for Optimal k')
plt.grid(True)
plt.show()


In [ ]:
from sklearn.cluster import KMeans

# Using t-SNE results with perplexity=30 for clustering
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_result = tsne.fit_transform(features)

# Perform K-means clustering with 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(tsne_result)

# Add cluster labels to the run_mean DataFrame
run_mean['Cluster'] = labels

# Visualize the clusters
plt.figure(figsize=(10, 7))
scatter = plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels, cmap='viridis', alpha=0.7)
plt.title('t-SNE with K-means Clustering (k=3)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.colorbar(scatter, orientation='horizontal', label='Cluster Label')
plt.grid(True)
plt.show()

# Analyze the clusters
cluster_summary = run_mean.groupby('Cluster')[['ACC', 'AVG_DIFFICULTY_LEVEL', 'MEDIAN_RT']].mean()
print(cluster_summary)

print('Cluster 0 (Cyan), Cluster 1 (Yellow), Cluster 2 (Purple)')

In [ ]:
# Describe the data to check for anomalies
print(run_mean.describe())

# Plot the features to check for anomalies
import seaborn as sns

sns.pairplot(run_mean, hue='Cluster')
plt.show()


In [ ]:
# Check for correlations
correlation_matrix = run_mean.corr()
print(correlation_matrix)

# Visualize the correlation matrix
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

# Drop one of each pair of highly correlated features if necessary
# For example, if ACC and AVG_DIFFICULTY_LEVEL are highly correlated, you might drop one.


In [ ]:
# Check for class imbalance
print(run_mean['Cluster'].value_counts())

# Visualize the class distribution
sns.countplot(x='Cluster', data=run_mean)
plt.title('Class Distribution')
plt.show()


# Single person connectivity matrix

# Autoencoder

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Assuming N_SUBJECTS and N_PARCELS are defined elsewhere

# Reshape fc array (choose your preferred method)
fc_reshaped = fc.reshape(N_SUBJECTS, -1)  # Flatten the entire connectivity matrix

# Define Autoencoder Model
class Autoencoder(tf.keras.Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Flatten(input_shape=(fc_reshaped.shape[1],)),  # Adjust for your reshaping method
      layers.Dense(128, activation="relu"),
      layers.Dense(64, activation="relu"),
      layers.Dense(latent_dim, activation="relu"),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(64, activation="relu"),
      layers.Dense(128, activation="relu"),
      layers.Dense(fc_reshaped.shape[1]),  # Adjust for your reshaping method
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

# Define Hyperparameters (adjust as needed)
latent_dim = 32  # Dimensionality of the latent space
epochs = 100
batch_size = 32

# Create the Autoencoder model
model = Autoencoder(latent_dim)

# Compile the model
model.compile(optimizer="adam", loss="mse")

# Train the model
model.fit(fc_reshaped, fc_reshaped, epochs=epochs, batch_size=batch_size)

# Extract Latent Features (optional)
latent_features = model.encoder.predict(fc_reshaped)
